In [1]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.40.68


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
sts_client = boto3.client('sts', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')
lambda_client = boto3.client('lambda', region_name='us-east-1')
bedrock_agent_client = boto3.client('bedrock-agent', region_name='us-east-1')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

[2025-11-21 13:48:23,687] p2072 {credentials.py:1364} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
session = boto3.session.Session()
region = 'us-east-1'
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '982248023567')

In [5]:
inference_profile = "us.amazon.nova-micro-v1:0"
foundation_model = inference_profile[3:]
foundation_model

'amazon.nova-micro-v1:0'

In [6]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
sts_client = boto3.client('sts', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')
lambda_client = boto3.client('lambda', region_name='us-east-1')
bedrock_agent_client = boto3.client('bedrock-agent', region_name='us-east-1')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

In [8]:
session = boto3.session.Session()
region = 'us-east-1'
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '982248023567')

In [9]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'
lambda_function_name

'hr-assistant-function-def-us-east-1-982248023567'

In [10]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year -3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y−%m−%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [11]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime

def get_available_vacations_days(employee_id):
    # Connect to the SQLite database
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:

        # Fetch the available vacation days for the employee
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]  # Unpack the tuple
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            conn.close()
            return available_vacation_days
        else:
            return_msg = f"No vacation data found for employed_id {employee_id}"
            print(return_msg)
            return return_msg
            conn.close()
    else:
        raise Exception(f"No employeed id provided")

    # Close the database connection
    conn.close()
    
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Connect to the SQLite database

    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            return_msg = f"Employee with ID {employee_id} does not exist."
            print(return_msg)
            conn.close()
            return return_msg

        # Check if the vacation days are available for the employee in the current year
        c.execute("SELECT employee_vacation_days_available FROM vacations WHERE employee_id = ? AND year = ?", (employee_id, current_year))
        available_days = c.fetchone()
        if available_days is None or available_days[0] < vacation_days:
            return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
            print(return_msg)
            conn.close()
            return return_msg

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        raise Exception(f"Error occurred: {e}")
        conn.rollback()
        # Close the database connection
        conn.close()
        return f"Error occurred: {e}"
        

def lambda_handler(event, context):
    original_db_file = 'employee_database.db'
    target_db_file = '/tmp/employee_database.db'
    if not os.path.exists(target_db_file):
        shutil.copy2(original_db_file, target_db_file)
    
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    
    if function == 'get_available_vacations_days':
        employee_id = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]

        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        vacation_days = get_available_vacations_days(employee_id)
        responseBody =  {
            'TEXT': {
                "body": f"available vacation days for employed_id {employee_id}: {vacation_days}"
            }
        }
    elif function == 'reserve_vacation_time':
        employee_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        if not start_date:
            raise Exception("Missing mandatory parameter: start_date")
        if not end_date:
            raise Exception("Missing mandatory parameter: end_date")
        
        completion_message = reserve_vacation_time(employee_id, start_date, end_date)
        responseBody =  {
            'TEXT': {
                "body": completion_message
            }
        }  
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


In [12]:
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'a13fb626-a237-4ebb-81c0-dc6a4b8be8a2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:48:35 GMT',
   'x-amzn-requestid': 'a13fb626-a237-4ebb-81c0-dc6a4b8be8a2',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [13]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

In [14]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [15]:

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '4f90e10f-69f0-4510-9e94-cff215a777b1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:48:46 GMT',
   'x-amzn-requestid': '4f90e10f-69f0-4510-9e94-cff215a777b1',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [16]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id, response

('DR1QERR4BI',
 {'ResponseMetadata': {'RequestId': '9b501544-b833-4ead-bb4b-15b085f3f913',
   'HTTPStatusCode': 202,
   'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:48:47 GMT',
    'content-type': 'application/json',
    'content-length': '692',
    'connection': 'keep-alive',
    'x-amzn-requestid': '9b501544-b833-4ead-bb4b-15b085f3f913',
    'x-amz-apigw-id': 'UaV3XEyVoAMEmcA=',
    'x-amzn-trace-id': 'Root=1-6920d02e-0e855e456f7b540a5bf1c928'},
   'RetryAttempts': 0},
  'agent': {'agentId': 'DR1QERR4BI',
   'agentName': 'hr-assistant-function-def',
   'agentArn': 'arn:aws:bedrock:us-east-1:982248023567:agent/DR1QERR4BI',
   'instruction': 'You are an HR agent, helping employees understand HR policies and manage vacation time',
   'agentStatus': 'CREATING',
   'foundationModel': 'us.amazon.nova-micro-v1:0',
   'description': 'Agent for providing HR assistance to manage vacation time',
   'orchestrationType': 'DEFAULT',
   'idleSessionTTLInSeconds': 1800,
   'agentResourceRoleArn': 'a

In [17]:

agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [18]:
agent_id

'DR1QERR4BI'

In [19]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [21]:

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
lambda_function_name
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'bf003b09-522c-4a80-9170-b9754446271e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:49:43 GMT',
   'content-type': 'application/json',
   'content-length': '1335',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bf003b09-522c-4a80-9170-b9754446271e',
   'x-amz-apigw-id': 'UaWANEFVIAMEGJg=',
   'x-amzn-trace-id': 'Root=1-6920d067-37450038385a17fc5f053650'},
  'RetryAttempts': 0},
 'agentActionGroup': {'agentId': 'DR1QERR4BI',
  'agentVersion': 'DRAFT',
  'actionGroupId': 'PYUDLSDIIX',
  'actionGroupName': 'VacationsActionGroup',
  'description': 'Actions for getting the number of available vacations days for an employee and confirm new time off',
  'createdAt': datetime.datetime(2025, 11, 21, 20, 49, 43, 499645, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2025, 11, 21, 20, 49, 43, 499645, tzinfo=tzutc()),
  'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:982248023567:function:hr-assistant-function-de

In [22]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

{'ResponseMetadata': {'RequestId': '58da1c67-010b-499c-a4a7-4d3515f1baa6',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:49:49 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '58da1c67-010b-499c-a4a7-4d3515f1baa6',
   'x-amz-apigw-id': 'UaWBIFiJIAMEk4w=',
   'x-amzn-trace-id': 'Root=1-6920d06d-081307515225fcf251613635'},
  'RetryAttempts': 0},
 'agentId': 'DR1QERR4BI',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 11, 21, 20, 49, 49, 560228, tzinfo=tzutc())}

In [23]:

response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId="DR1QERR4BI"
)

response

{'ResponseMetadata': {'RequestId': '5ed2bbbb-f307-4e36-af33-b526059ac058',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 21 Nov 2025 20:49:57 GMT',
   'content-type': 'application/json',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5ed2bbbb-f307-4e36-af33-b526059ac058',
   'x-amz-apigw-id': 'UaWCWFKqIAMETAg=',
   'x-amzn-trace-id': 'Root=1-6920d075-195bd3ce24bb61445f74b808'},
  'RetryAttempts': 0},
 'agentAlias': {'agentId': 'DR1QERR4BI',
  'agentAliasId': '7IHRHWU7D2',
  'agentAliasName': 'test-alias-1',
  'agentAliasArn': 'arn:aws:bedrock:us-east-1:982248023567:agent-alias/DR1QERR4BI/7IHRHWU7D2',
  'routingConfiguration': [{}],
  'createdAt': datetime.datetime(2025, 11, 21, 20, 49, 57, 200516, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2025, 11, 21, 20, 49, 57, 200516, tzinfo=tzutc()),
  'agentAliasStatus': 'CREATING',
  'aliasInvocationState': 'ACCEPT_INVOCATIONS'}}

In [24]:
## create a random id for session initiator id
agent_alias_id="7IHRHWU7D2"
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-21 13:50:13,314] p2072 {2867722568.py:17} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 21 Nov 2025 20:50:13 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f36e43d4-715e-4fc2-823e-a86e6bed2926'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f36e43d4-715e-4fc2-823e-a86e6bed2926',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x107b22570>,
 'contentType': 'application/json',
 'sessionId': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715'}


[2025-11-21 13:50:16,137] p2072 {2867722568.py:24} INFO - Final answer ->
The employee with employee_id 1 has 25 vacation days available.


In [32]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Great. please reserve one day of time off for the employee with employee_id set to 1 for Dec 28, 29, 30 of 2025",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-21 14:00:36,893] p2072 {1940276742.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 21 Nov 2025 21:00:36 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '30ab9a9a-066d-46ed-8c88-da05ff7898eb'},
                      'HTTPStatusCode': 200,
                      'RequestId': '30ab9a9a-066d-46ed-8c88-da05ff7898eb',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x104d24740>,
 'contentType': 'application/json',
 'sessionId': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715'}


[2025-11-21 14:00:43,247] p2072 {1940276742.py:17} INFO - Final answer ->
The vacation times for employee with ID 1 from Dec 28 to Dec 30, 2025, have been reserved successfully.


In [34]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="What days does employee with employee_id set to 1 have off?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-21 14:01:48,215] p2072 {216679380.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 21 Nov 2025 21:01:48 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'a67d2328-d61c-40d5-9159-d24538f23728'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a67d2328-d61c-40d5-9159-d24538f23728',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x108322f30>,
 'contentType': 'application/json',
 'sessionId': 'ad0ce20c-c71b-11f0-ab3f-784f434d5715'}


[2025-11-21 14:01:50,216] p2072 {216679380.py:17} INFO - Final answer ->
<outOfDomain reason="The available tools do not provide specific days off for an employee."></outOfDomain>
